In [2]:
from tasks.mm_tasks.snli_ve import SnliVeTask

/root/anaconda3/envs/acmmm/lib/python3.7/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2022-06-18 17:21:14.833977: W tensorflow/stream_executor/platform/default/dso_loader.cc:59] Could not load dynamic library 'libcudart.so.10.1'; dlerror: libcudart.so.10.1: cannot open shared object file: No such file or directory
2022-06-18 17:21:14.834011: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [1]:
import torch
import numpy as np
from fairseq import checkpoint_utils, options, tasks, utils
from fairseq.dataclass.utils import convert_namespace_to_omegaconf
from tasks.mm_tasks.snli_ve import SnliVeTask

# from models.ofa import OFAModel
from PIL import Image

torch.cuda.set_device(0)

# specify some options for evaluation
parser = options.get_generation_parser()
input_args = ["", 
"--task=snli_ve",
"--path=checkpoints/snli_ve_large_best.pt",
"--bpe-dir=utils/BPE", 
"--batch-size=8 "]
args = options.parse_args_and_arch(parser, input_args)
cfg = convert_namespace_to_omegaconf(args)
# cfg.task['data'] = '/root/thesis/ofa/dataset/snli_ve_data/snli_ve_test.tsv'
# cfg.task['selected_cols'] = '0,2,3,4,5'


/root/anaconda3/envs/acmmm/lib/python3.7/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2022-06-18 17:00:36.711983: W tensorflow/stream_executor/platform/default/dso_loader.cc:59] Could not load dynamic library 'libcudart.so.10.1'; dlerror: libcudart.so.10.1: cannot open shared object file: No such file or directory
2022-06-18 17:00:36.712009: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [2]:
# overrides = eval(cfg.common_eval.model_overrides)
overrides = {
    'data': '/root/thesis/ofa/dataset/snli_ve_data/snli_ve_test.tsv', 
    'bpe_dir': '/root/thesis/ofa/utils/BPE', 
    # 'selected_cols': '0,2,3,4,5'
    }

In [3]:
# task = tasks.setup_task(cfg.task)
models, saved_cfg, task = checkpoint_utils.load_model_ensemble_and_task(
    utils.split_paths(cfg.common_eval.path),
    arg_overrides=overrides,
    # task=task
)

2022-06-18 13:49:30 | INFO | tasks.ofa_task | source dictionary: 59457 types
2022-06-18 13:49:30 | INFO | tasks.ofa_task | target dictionary: 59457 types


In [4]:
# task.load_dataset(cfg.dataset.gen_subset, task_cfg=saved_cfg.task)

In [4]:
# Image transform
from torchvision import transforms
from utils.trie import Trie
from data import data_utils

import re
mean = [0.5, 0.5, 0.5]
std = [0.5, 0.5, 0.5]

patch_resize_transform = transforms.Compose([
    lambda image: image.convert("RGB"),
    transforms.Resize((task.cfg.patch_image_size, task.cfg.patch_image_size), interpolation=Image.BICUBIC),
    transforms.ToTensor(),
    transforms.Normalize(mean=mean, std=std),
])

bos_item = torch.LongTensor([task.src_dict.bos()])
eos_item = torch.LongTensor([task.src_dict.eos()])
pad_idx = task.src_dict.pad()
eos_idx = task.src_dict.eos()
tgt_dict = task.tgt_dict
max_src_length = 80

device='cuda'

def encode_text(text, length=None, append_bos=False, append_eos=False):
    line = [
      task.bpe.encode(' {}'.format(word.strip())) 
      if not word.startswith('<code_') and not word.startswith('<bin_') else word
      for word in text.strip().split()
    ]
    line = ' '.join(line)
    s = task.tgt_dict.encode_line(
        line=line,
        add_if_not_exist=False,
        append_eos=False
    ).long()
    if length is not None:
        s = s[:length]
    if append_bos:
        s = torch.cat([bos_item, s])
    if append_eos:
        s = torch.cat([s, eos_item])
    return s

def pre_caption(caption, max_words):
    caption = caption.lower().lstrip(",.!?*#:;~").replace('-', ' ').replace('/', ' ').replace('<person>', 'person')

    caption = re.sub(
        r"\s{2,}",
        ' ',
        caption,
    )
    caption = caption.rstrip('\n')
    caption = caption.strip(' ')

    # truncate caption
    caption_words = caption.split(' ')
    if len(caption_words) > max_words:
        caption = ' '.join(caption_words[:max_words])

    return caption

def construct_sample(uniq_id ,image, hypothesis, caption,label):
    if label == 'contradiction':
        label = 'no'
    elif label == 'entailment':
        label = 'yes'
    elif label == 'neutral':
        label = 'maybe'
    else:
        raise NotImplementedError

    patch_image = patch_resize_transform(image)
    patch_mask = torch.tensor([True])

    hypothesis = pre_caption(hypothesis, max_src_length)
    src_item = encode_text(' does the image describe " {} "?'.format(hypothesis))
    tgt_item = encode_text(" {}".format(label))
    ref_dict = {label: 1.0}

    # caption = pre_caption(caption, max_src_length)
    # src_item = encode_text(' can image and text1 " {} " imply text2 " {} "?'.format(caption, hypothesis))

    src_item = torch.cat([bos_item, src_item, eos_item])

    prev_output_item = torch.cat([src_item[:-1], tgt_item])
    target_item = torch.cat([prev_output_item[1:], eos_item])
    decoder_prompt = src_item[:-1]
    target_item[:-len(tgt_item)-1] = 1 # tgt_dict.pad()

    constraint_trie = Trie(tgt_dict.eos())
    constraint_mask = torch.zeros((len(target_item), len(tgt_dict))).bool()
    start_idx = len(target_item) - len(tgt_item) - 1
    for i in range(len(target_item)-len(tgt_item)-1, len(target_item)):
        constraint_prefix_token = [tgt_dict.bos()] + target_item[start_idx:i].tolist()
        constraint_nodes = constraint_trie.get_next_layer(constraint_prefix_token)
        constraint_mask[i][constraint_nodes] = True
    example = {
            "id": uniq_id,
            "source": src_item.to(device),#
            "patch_image": patch_image.to(device),#
            "patch_mask": patch_mask.to(device),#
            "target": target_item.to(device),
            "prev_output_tokens": prev_output_item.to(device),
            "decoder_prompt": decoder_prompt.to(device),
            "ref_dict": ref_dict,
            "constraint_mask": constraint_mask.to(device)
        }
    return example

/root/anaconda3/envs/ofa/lib/python3.7/site-packages/ipykernel_launcher.py:12: DeprecationWarning: BICUBIC is deprecated and will be removed in Pillow 10 (2023-07-01). Use Resampling.BICUBIC instead.
  if sys.path[0] == '':
/root/anaconda3/envs/ofa/lib/python3.7/site-packages/torchvision/transforms/transforms.py:281: UserWarning: Argument interpolation should be of type InterpolationMode instead of int. Please, use InterpolationMode enum.
  "Argument interpolation should be of type InterpolationMode instead of int. "


In [5]:
from utils.eval_utils import eval_snli_ve
from data.mm_data.snli_ve_dataset import collate

In [6]:
device = 'cuda'
models[0].to(device);

In [7]:
torch.manual_seed(0)
with torch.no_grad():
    d0 = construct_sample("I1",Image.open('/root/thesis/dataset/public_test_mmsys/661.jpg'), 
    'The Red Cross burial workers carry a box containing the body of an 11-month-old girl', '','entailment')
    sample = collate([d0,d0,d0,d0,d0], pad_idx, eos_idx)
    r = eval_snli_ve(task,None,models,sample)

/root/anaconda3/envs/ofa/lib/python3.7/site-packages/torch/nn/functional.py:718: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  /pytorch/c10/core/TensorImpl.h:1156.)
  return torch.max_pool2d(input, kernel_size, stride, padding, dilation, ceil_mode)


In [8]:
r

([{'uniq_id': 'I1', 'answer': 'maybe', 'e_score': tensor(0.0059)},
  {'uniq_id': 'I1', 'answer': 'maybe', 'e_score': tensor(0.0044)},
  {'uniq_id': 'I1', 'answer': 'maybe', 'e_score': tensor(0.0121)},
  {'uniq_id': 'I1', 'answer': 'maybe', 'e_score': tensor(0.0044)},
  {'uniq_id': 'I1', 'answer': 'maybe', 'e_score': tensor(0.0210)}],
 [0, 0, 0, 0, 0])

In [7]:
# CONTRADICT / ENTAILMENT / NEUTRAL
import pandas as pd
from tqdm import tqdm
import os
data_path = '/root/thesis/experiment/full_test.csv'
df = pd.read_csv(data_path)

In [8]:
def create_sample(row):
    # print(row)
    with torch.no_grad():
        d1 = construct_sample(row["img_local_path"]+'__c1',
            Image.open(os.path.join('/root/thesis/dataset/',row["img_local_path"])), 
            row['caption1'], '','entailment')
        d2 = construct_sample(row["img_local_path"]+'__c2',
            Image.open(os.path.join('/root/thesis/dataset/',row["img_local_path"])), 
            row['caption2'], '','entailment')
        sample = collate([d1,d2], pad_idx, eos_idx)
        result = eval_snli_ve(task,None,models,sample)
    row['c1_entail'] = result[0][0]['e_score'].numpy().tolist()
    row['c2_entail'] = result[0][1]['e_score'].numpy().tolist()
    return row

In [9]:
torch.manual_seed(0)
tqdm.pandas()
z=df.progress_apply(lambda x: create_sample(x), axis=1)

  0%|          | 0/1700 [00:00<?, ?it/s]/root/thesis/ofa/data/mm_data/snli_ve_dataset.py:61: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  decoder_prompts = np.array([s['decoder_prompt'].tolist() for s in samples])
/root/anaconda3/envs/ofa/lib/python3.7/site-packages/torch/nn/functional.py:718: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  /pytorch/c10/core/TensorImpl.h:1156.)
  return torch.max_pool2d(input, kernel_size, stride, padding, dilation, ceil_mode)
100%|██████████| 1700/1700 [20:37<00:00,  1.37it/s]


In [31]:
z[(z['c2_entail']>0.4) | (z['c1_entail']>0.4)]

,img_local_path,caption1,caption2,context_label,article_url,maskrcnn_bboxes,caption1_modified,caption1_entities,caption2_modified,caption2_entities,bert_base_score,bert_large_score,c1_entail,c2_entail
55,public_test_mmsys/55.jpg,Trash being left behind by the migrant caravan.,This photograph was not recent and did not sho...,1,https://www.snopes.com/fact-check/refugee-cara...,"[[492.66387939453125, 337.4176940917969, 819.8...",Trash being left behind by the migrant caravan.,[],This photograph was not recent and did not sho...,"[['Honduras', 'GPE'], ['U.S', 'GPE'], ['Mexico...",0.339660,0.161456,0.417470,0.042547
92,public_test_mmsys/92.jpg,Zurich's Bahnhofstrasse,Cars in the street,0,https://www.bbc.com/news/in-pictures-54175441,"[[708.0590209960938, 246.20138549804688, 999.0...",GPE's ORG,"[['Zurich', 'GPE']]",Cars in the street,[],0.029287,0.106145,0.053713,0.778402
105,public_test_mmsys/105.jpg,Lamar preforming in 2013.,Kendrick Lamar in concert.,0,https://en.wikipedia.org/wiki/Kendrick_Lamar#/...,"[[689.6602172851562, 0.0, 1198.6527099609375, ...",PERSON preforming in DATE.,"[['Lamar', 'GPE'], ['2013', 'DATE']]",PERSON in concert.,"[['Kendrick Lamar', 'PERSON']]",0.321324,0.428131,0.053281,0.538456
107,public_test_mmsys/107.jpg,While community leader Janet Sharty hands out ...,Children smile holding their sweets.,0,https://www.bbc.com/news/world-africa-55573602,"[[563.9031982421875, 102.29672241210938, 890.0...",While community leader PERSON hands out popcor...,"[['Janet Sharty', 'PERSON'], [""New Year's"", 'E...",Children smile holding their sweets.,[],0.530860,0.521051,0.009720,0.420947
127,public_test_mmsys/127.jpg,A tree in snow,An old tree in winter at the Osogovo Mountain ...,0,https://www.bbc.com/news/in-pictures-56211135,"[[871.3872680664062, 616.6320190429688, 891.98...",A tree in snow,[],"An old tree in DATE at LOC near GPE, ORG","[['winter', 'DATE'], ['the Osogovo Mountain', ...",0.573069,0.514833,0.844991,0.063245
144,public_test_mmsys/144.png,"On Feb. 19, 2021, U.S. President Joe Biden utt...","Biden did not use the N-word in that moment, n...",1,https://www.snopes.com/fact-check/joe-biden-n-...,"[[307.34832763671875, 148.58619689941406, 490....","On DATE, GPE President PERSON uttered a racial...","[['Feb. 19, 2021', 'DATE'], ['U.S.', 'GPE'], [...","Biden did not use the N-word in that moment, n...",[],0.119273,0.193871,0.006668,0.486215
158,public_test_mmsys/158.jpg,Former New York City Mayor Michael Bloomberg c...,If Bloomberg were to divide his wealth equally...,1,https://www.snopes.com/fact-check/bloomberg-po...,"[[322.7524108886719, 369.62060546875, 392.5690...",Former GPE Mayor PERSON could have given each ...,"[['New York City', 'GPE'], ['Michael Bloomberg...",If PERSON were to divide his wealth equally am...,"[['Bloomberg', 'PERSON'], ['American', 'NORP']...",0.440843,0.611205,0.020918,0.456249
169,public_test_mmsys/169.jpg,A woman walks her dog on a Paris bridge,The Eiffel Tower seen in background,0,https://english.aawsat.com/home/article/230781...,"[[322.5820617675781, 85.41505432128906, 469.61...",A woman walks her dog on a GPE bridge,"[['Paris', 'GPE']]",FAC seen in background,[],-0.075973,0.169833,0.077998,0.710767
172,public_test_mmsys/172.jpg,The legal drinking age has been raised to 23.,Image shows flaming drinks,1,https://www.snopes.com/fact-check/legal-drinki...,"[[61.6138801574707, 212.6248016357422, 269.531...",The legal drinking age has been raised to CARD...,"[['23', 'CARDINAL']]",Image shows flaming drinks,[],0.185332,0.411020,0.220466,0.778446
187,public_test_mmsys/187.jpg,"Students in Lausanne, Switzerland, take part i...",Students dressed as astronauts,0,https://www.bbc.com/news/in-pictures-56442714,"[[97.21846008300781, 56.18781280517578, 337.31...","Students in GPE, GPE, take part in a lesson to...","[['Lausanne', 'GPE'], ['Switzerland', 'GPE'], ...",Students dressed as astronauts,[],0.479297,0.467664,0.001146,0.529805


In [10]:
z.to_csv('ofa_full.csv', index=False)

In [30]:
l = []
for g in z:
    l.append(g[0])
    l.append(g[1])

In [33]:
split_l = np.array_split(l,200)

In [34]:
c1=[]
c2=[]

for l_ in tqdm(split_l):
    with torch.no_grad():
        sample = collate(l_, pad_idx, eos_idx)
        result = eval_snli_ve(task,None,models,sample)

    for r in result[0]:
        img_local_path,cap = r['uniq_id'].split('__')
        if cap=='c1':
            c1.append(r['e_score'].numpy().tolist())
        else:
            c2.append(r['e_score'].numpy().tolist())

  0%|          | 0/200 [00:00<?, ?it/s]


RuntimeError: CUDA out of memory. Tried to allocate 570.00 MiB (GPU 0; 39.41 GiB total capacity; 21.10 GiB already allocated; 530.56 MiB free; 21.54 GiB reserved in total by PyTorch)

In [10]:
next(models[0].parameters()).is_cuda

False